In [1]:
import csv
import pickle
import PySimpleGUI as sg
from termcolor import colored
import pandas as pd
import numpy as np
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import nltk 
from nltk.stem import PorterStemmer
ps = PorterStemmer()
from autocorrect import Speller
spell = Speller(lang='en')
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from PyDictionary import PyDictionary
dictionary=PyDictionary()
stop = stopwords.words('english')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\giorg\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\giorg\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Import the dataframe

In [2]:
pd.set_option('display.max_colwidth', None)
df = pd.read_csv(r'C:\Users\giorg\Information Retrieval\Final_Coventry_Univerity_papers_clean.csv', encoding = 'utf-8-sig')

# Clean the columns of the dataframe

In [3]:
# convert columns into strings

df['descriptions']=df['descriptions'].apply(str)
df['paper_titles']=df['paper_titles'].apply(str)
df['authors']=df['authors'].apply(str)

In [4]:
# dropping the duplicate papers

df = df.drop_duplicates()

print('The remaining papers are: ',len(df))

The remaining papers are:  31545


In [5]:
# Removing the space from headers

df.columns = df.columns.str.strip()

In [6]:
# Remove special characters

df['descriptions'] = df['descriptions'].map(lambda x: re.sub(r"[^\w+( +\w+)*$]", '', x))
df['paper_titles'] = df['paper_titles'].map(lambda x: re.sub(r"[^\w+( +\w+)*$]", '', x))
df['authors'] = df['authors'].map(lambda x: re.sub(r"[^\w+( +\w+)*$]", '', x))

In [7]:
# Turn them into lowercase

df['descriptions'] = df['descriptions'].str.lower()
df['paper_titles'] = df['paper_titles'].str.lower()
df['authors'] = df['authors'].str.lower()

In [8]:
# Remove the doubled spaces

df['descriptions'] = df['descriptions'].map(lambda x:  re.sub(r'\s{2,}', ' ', x))
df['paper_titles'] = df['paper_titles'].map(lambda x:  re.sub(r'\s{2,}', ' ', x))
df['authors'] = df['authors'].map(lambda x:  re.sub(r'\s{2,}', ' ', x))

In [9]:
# Remove Mentions

df['descriptions'] = df['descriptions'].map(lambda x:  re.sub(r'@\w+', '', x))
df['paper_titles'] = df['paper_titles'].map(lambda x:  re.sub(r'@\w+', '', x))
df['authors'] = df['authors'].map(lambda x: re.sub(r'@\w+', '', x))

In [10]:
# Remove Unicode

df['descriptions'] = df['descriptions'].map(lambda x:  re.sub(r'[^\x00-\x7F]+', ' ', x))
df['paper_titles'] = df['paper_titles'].map(lambda x:  re.sub(r'[^\x00-\x7F]+', ' ', x))
df['authors'] = df['authors'].map(lambda x: re.sub(r'[^\x00-\x7F]+', ' ', x))

In [11]:
# Remove punctuations

df['descriptions'] = df['descriptions'].map(lambda x:  re.sub(r'[%s]' % re.escape(string.punctuation), ' ', x))
df['paper_titles'] = df['paper_titles'].map(lambda x:  re.sub(r'[%s]' % re.escape(string.punctuation), ' ', x))
df['authors'] = df['authors'].map(lambda x: re.sub(r'[%s]' % re.escape(string.punctuation), ' ', x))

In [12]:
# Remove stopwords

df['descriptions'] = df['descriptions'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['paper_titles'] = df['paper_titles'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
df['authors'] = df['authors'].map(lambda x:  re.sub(r'\s{2,}', ' ', x))

In [13]:
paper_titles_list = df['paper_titles'].tolist()
authors_list = df['authors'].tolist()
descriptions_list = df['descriptions'].tolist()

# Create the inverted index for paper titles

In [14]:
list_titles = []

for i in range(len(paper_titles_list)):
    list_titles.append(paper_titles_list[i].split(' '))
list_titles

[['sonochemistry'],
 ['applied',
  'sonochemistry',
  'uses',
  'power',
  'ultrasound',
  'chemistry',
  'processing'],
 ['uses', 'ultrasound', 'food', 'technology'],
 ['practical', 'sonochemistry'],
 ['ultrasound', 'synthetic', 'organic', 'chemistry'],
 ['investigation',
  'effects',
  'ultrasound',
  'vegetal',
  'tissues',
  'solvent',
  'extraction'],
 ['microwave', 'ultrasonic', 'processing', 'realistic', 'option', 'industry'],
 ['quantifying', 'sonochemistry', 'casting', 'light', 'black', 'art'],
 ['sonochemistry', 'part', '1the', 'physical', 'aspects'],
 ['potential',
  'use',
  'ultrasound',
  'extraction',
  'antioxidants',
  'rosmarinus',
  'officinalis',
  'food',
  'pharmaceutical',
  'industry'],
 ['effect',
  'ultrasound',
  'treatment',
  'solubility',
  'foaming',
  'properties',
  'whey',
  'protein',
  'suspensions'],
 ['development',
  'evaluation',
  'ultrasound',
  'treatment',
  'bacterial',
  'suspensions',
  'study',
  'frequency',
  'power',
  'sonication',
  

In [15]:
unique_paper_titles = []

for title in list_titles:
    for word in title:
        if word not in unique_paper_titles:
            unique_paper_titles.append(word)
print(unique_paper_titles) 

['sonochemistry', 'applied', 'uses', 'power', 'ultrasound', 'chemistry', 'processing', 'food', 'technology', 'practical', 'synthetic', 'organic', 'investigation', 'effects', 'vegetal', 'tissues', 'solvent', 'extraction', 'microwave', 'ultrasonic', 'realistic', 'option', 'industry', 'quantifying', 'casting', 'light', 'black', 'art', 'part', '1the', 'physical', 'aspects', 'potential', 'use', 'antioxidants', 'rosmarinus', 'officinalis', 'pharmaceutical', 'effect', 'treatment', 'solubility', 'foaming', 'properties', 'whey', 'protein', 'suspensions', 'development', 'evaluation', 'bacterial', 'study', 'frequency', 'sonication', 'time', 'cultured', 'bacillus', 'species', 'treated', 'soy', 'proteins', 'applications', 'dosimetry', 'aqueous', 'terephthalate', 'ion', 'fluorescence', 'monitor', 'bioactive', 'principles', 'plant', 'materials', 'processingthe', 'way', 'forward', 'rutin', 'flower', 'buds', 'sophora', 'japonica', 'environmentproviding', 'green', 'link', 'physics', 'engineering', 'sono

In [16]:
dict_titles = {}
for word in unique_paper_titles:
    for i, title in enumerate(list_titles):
        if word in title:
            if word in dict_titles.keys():
                dict_titles[word].append(i)
            else:
                dict_titles[word]= [i]

In [17]:
dict_titles

{'sonochemistry': [0,
  1,
  3,
  7,
  8,
  13,
  15,
  20,
  21,
  24,
  29,
  37,
  39,
  48,
  56,
  59,
  67,
  76,
  91,
  108,
  115,
  123,
  144,
  158,
  179,
  183,
  188,
  192,
  197,
  211,
  212,
  215,
  227,
  236,
  240,
  253,
  273,
  274,
  286,
  299,
  300,
  303,
  351,
  364,
  375,
  386,
  387,
  388,
  390,
  434,
  436,
  450,
  459,
  466,
  475,
  497],
 'applied': [1,
  161,
  389,
  1469,
  1708,
  2677,
  2881,
  5924,
  6089,
  6254,
  6384,
  6809,
  7108,
  7400,
  7635,
  7857,
  8482,
  8638,
  8840,
  9690,
  10021,
  10353,
  10376,
  11369,
  11513,
  11516,
  11543,
  13878,
  13949,
  13951,
  14421,
  14915,
  15417,
  16003,
  16005,
  16768,
  17065,
  17107,
  17129,
  18995,
  19034,
  19807,
  20721,
  21038,
  21083,
  21265,
  22454,
  23443,
  23444,
  23629,
  23887,
  24323,
  24511,
  24526,
  24999,
  25120,
  25453,
  25686,
  26064,
  26078,
  26495,
  27204,
  27892,
  28312,
  28892,
  29048,
  30247],
 'uses': [1,
  2,
  13,


# Create the inverted index for authors

In [18]:
list_authors = []

for i in range(len(authors_list)):
    list_authors.append(authors_list[i].split(' '))
list_authors

[['timothy', 'james', 'mason', 'j', 'phillip', 'lorimer'],
 ['timothy', 'j', 'mason', 'j', 'phillip', 'lorimer'],
 ['timothy', 'j', 'mason', 'larysa', 'paniwnyk', 'jp', 'lorimer'],
 ['timothy', 'j', 'mason', 'dietmar', 'peters'],
 ['timothy', 'j', 'mason'],
 ['maricela',
  'toma',
  'm',
  'vinatoru',
  'larysa',
  'paniwnyk',
  'timothy',
  'j',
  'mason'],
 ['cristina', 'leonelli', 'timothy', 'j', 'mason'],
 ['tj', 'mason', 'jp', 'lorimer', 'dm', 'bates'],
 ['john', 'p', 'lorimer', 'timothy', 'j', 'mason'],
 ['s',
  'albu',
  'e',
  'joyce',
  'larysa',
  'paniwnyk',
  'jp',
  'lorimer',
  'timothy',
  'j',
  'mason'],
 ['anet',
  're',
  'ek',
  'jambrak',
  'timothy',
  'j',
  'mason',
  'vesna',
  'lelas',
  'zoran',
  'herceg',
  'ivana',
  'ljubi',
  'herceg'],
 ['e', 'joyce', 'ss', 'phull', 'jp', 'lorimer', 'tj', 'mason'],
 ['anet',
  're',
  'ek',
  'jambrak',
  'vesna',
  'lelas',
  'timothy',
  'j',
  'mason',
  'greta',
  'kre',
  'i',
  'marija',
  'badanjak'],
 ['timothy'

In [19]:
unique_authors = []

for title in list_authors:
    for word in title:
        if word not in unique_authors:
            unique_authors.append(word)
print(unique_authors) 

['timothy', 'james', 'mason', 'j', 'phillip', 'lorimer', 'larysa', 'paniwnyk', 'jp', 'dietmar', 'peters', 'maricela', 'toma', 'm', 'vinatoru', 'cristina', 'leonelli', 'tj', 'dm', 'bates', 'john', 'p', 's', 'albu', 'e', 'joyce', 'anet', 're', 'ek', 'jambrak', 'vesna', 'lelas', 'zoran', 'herceg', 'ivana', 'ljubi', 'ss', 'phull', 'greta', 'kre', 'i', 'marija', 'badanjak', 'malcolm', 'jw', 'povey', 'yl', 'zhao', 'otilia', 'radu', 'pi', 'filip', 'd', 'lazurca', 'beaufoy', 'bao', 'yang', 'mouming', 'shi', 'ning', 'yueming', 'jiang', 'ruo', 'feng', 'yiyun', 'changping', 'zhu', 'tinghe', 'yu', 'zhibiao', 'wang', 'dj', 'walton', 'veronica', 'ez', 'lindley', 'aj', 'cobley', 'je', 'graves', 'morgan', 'calinescu', 'ap', 'newman', 'b', 'pollet', 'sali', 'ov', 'tc', 'cuthbert', 'ea', 'brookfield', 'lawrence', 'a', 'crum', 'jacques', 'l', 'reisse', 'kenneth', 'suslick', 'enrique', 'riera', 'antonio', 'vercet', 'pascual', 'lopezbuesa', 'berlan', 'ilana', 'perelshtein', 'elena', 'ruderman', 'nina', 'pe

In [20]:
dict_authors = {}
for word in unique_authors:
    for i, title in enumerate(list_authors):
        if word in title:
            if word in dict_authors.keys():
                dict_authors[word].append(i)
            else:
                dict_authors[word]= [i]

# Create the inverted index for Descriptions

In [21]:
# list_descriptions = []

# for i in range(len(descriptions_list)):
#     list_descriptions.append(descriptions_list[i].split(' '))
# list_descriptions

In [22]:
# unique_descriptions = []

# for title in list_descriptions:
#     for word in title:
#         if word not in unique_descriptions:
#             unique_descriptions.append(word)
# print(unique_descriptions) 

In [23]:
# dict_descriptions = {}
# for word in unique_descriptions:
#     for i, title in enumerate(list_descriptions):
#         if word in title:
#             if word in dict_descriptions.keys():
#                 dict_descriptions[word].append(i)
#             else:
#                 dict_descriptions[word]= [i]

# Search Engine

In [24]:
class color:
   PURPLE = '\033[95m'
   CYAN = '\033[96m'
   DARKCYAN = '\033[36m'
   BLUE = '\033[94m'
   GREEN = '\033[92m'
   YELLOW = '\033[93m'
   RED = '\033[91m'
   BOLD = '\033[1m'
   UNDERLINE = '\033[4m'
   END = '\033[0m'


In [44]:
class SearchEngine:

    def SearchingTitle(self):
        query = input('Enter your query : ')
        
        # Autocorrect our query
        #query = spell(query)
        
        # Stemming the query
        #query = ps.stem(query)
        
        # Getting synonyms
        #query=dictionary.synonym(query)
        
        # Remove special characters
        query = re.sub(r"[^\w+( +\w+)*$]", '', query)

        # convert it into lowercase
        query = query.lower() 

        # Remove Mentions
        query = re.sub(r'@\w+', '', query)

        # Remove Unicode
        query = re.sub(r'[^\x00-\x7F]+', ' ', query)
        
        # Remove stopwords
        query = query.split()
        query = [word for word in query if not word in stopwords.words()]
        
        print('')
        print('')
        print('')
        print('')
        
        title_score ={}
        all_titles = []
        
        for word in query:
            if word not in dict_titles.keys():
                print(color.BOLD +'Query not in the database'+color.END)
            else:
                for word in query:
                    all_titles += dict_titles[word]

                all_titles = list(set(all_titles))

                for doc in all_titles:
                     title_score[doc] = 0
                for word in query:
                    for doc in dict_titles[word]:
                        title_score[doc] += 1
                print(title_score)
                print('')
                print('')
                title_score_sort = dict(sorted(title_score.items(), key=lambda item: item[1]))
                print(colored('The related Papers are:','green', attrs=['bold']))
                print('')
                print('')
                for doc in title_score_sort:
                    print(df.loc[doc])
                    print('')
                    print('')
                    
#############################################################################################################################            

    def SearchingAuthor(self):
        query = input('Enter your query : ')

        #Autocorrect our query
        #query = spell(query)
        
        # Remove special characters
        query = re.sub(r"[^\w+( +\w+)*$]", '', query)
        
        # convert it into lowercase
        query = query.lower()
        
        # Remove Mentions
        query = re.sub(r'@\w+', '', query)
        
        # Remove Unicode
        query = re.sub(r'[^\x00-\x7F]+', ' ', query)
        
        # Remove stopwords
        query = query.split()
        query = [word for word in query if not word in stopwords.words()]
        
        print('')
        print('')
        print('')
        print('')

        author_score ={}
        all_authors = []
        for word in query:
            if word not in dict_titles.keys():
                print(color.BOLD +'Query not in the database'+color.END)
            else:
                for word in query:
                    all_authors += dict_authors[word]

                all_authors = list(set(all_authors))

                for doc in all_authors:
                    author_score[doc] = 0
                for word in query:
                    for doc in dict_authors[word]:
                        author_score[doc] += 1

                author_score_sort = dict(sorted(author_score.items(), key=lambda item: item[1]))
                print(colored('The related Papers are:','green', attrs=['bold']))
                print('')
                print('')
                for doc in author_score:
                    print((df.loc[doc]))
                    print('')
                    print('')

#############################################################################################################################            
                    

    def SearchingDate(self):
        query = input('Enter your query : ')
        print('')
        print('')
        file = csv.reader(open('C:/Users/giorg/Information Retrieval/Final_Coventry_Univerity_papers_clean.csv', 'r', encoding="utf-8-sig"))
        print(colored('The related Papers are:','green', attrs=['bold']))
        print('')
        print('')
        
        for row in file:
            if query in row[2]:
                print('')
                print(row)
                
#############################################################################################################################            

    def SearchingDescription(self):
        query = input('Enter your query : ')
        file = csv.reader(open('C:/Users/giorg/Information Retrieval/Final_Coventry_Univerity_papers_clean.csv', 'r', encoding="utf-8-sig"))
        # Autocorrect our query
        #query = spell(query)
        
        # Stemming the query
        query = ps.stem(query)
        
        # Getting synonyms
        #query=dictionary.synonym(query)
        
        # Remove special characters
        query = re.sub(r"[^\w+( +\w+)*$]", '', query)

        # convert it into lowercase
        query = query.lower() 

        # Remove Mentions
        query = re.sub(r'@\w+', '', query)

        # Remove Unicode
        query = re.sub(r'[^\x00-\x7F]+', ' ', query)
        
        # Remove stopwords
        query = query.split()
        query = [word for word in query if not word in stopwords.words()]
        
        print('')
        print('')
        print(colored('The related Papers are:','green', attrs=['bold']))
        print('')
        print('')
        
        for word in query:
            for row in file:
                if word in row[3]:
                    print(row)
                    print('')
#         descriptions_score ={}
#         all_descriptions = []
        
#         for word in query:
#             if word not in dict_descriptions.keys():
#                 print(color.BOLD +'Query not in the database'+color.END)
#             else:
#                 for word in query:
#                     all_descriptions += dict_descriptions[word]

#                 all_descriptions = list(set(all_descriptions))

#                 for doc in all_descriptions:
#                      descriptions_score[doc] = 0
#                 for word in query:
#                     for doc in dict_titles[word]:
#                         descriptions_score[doc] += 1
#                 print(descriptions_score)
#                 print('')
#                 print('')
#                 descriptions_score_sort = dict(sorted(descriptions_score.items(), key=lambda item: item[1]))
#                 print(colored('The related Papers are:','green', attrs=['bold']))
#                 print('')
#                 print('')
#                 for doc in descriptions_score_sort:
#                     print(df.loc[doc])
#                     print('')
#                     print('')
                    
#############################################################################################################################            

print(color.BOLD +'This is the Coventry University Search Engine'+ color.END)
print('')

print('Enter 1 to get Papers regarding their Title')
print('Enter 2 to get Papers regarding their Authors')
print('Enter 3 to get Papers regarding their Date')
print('Enter 4 to get Papers regarding their Description')



src = int(input(color.BLUE +'Enter your option: '+ color.END))
a = SearchEngine()

if src == 1:
    a.SearchingTitle()

if src == 2:
    a.SearchingAuthor()

if src == 3:
    a.SearchingDate()

if src == 4:
    a.SearchingDescription()

else:
    print('')


This is the Coventry University Search Engine

Enter 1 to get Papers regarding their Title
Enter 2 to get Papers regarding their Authors
Enter 3 to get Papers regarding their Date
Enter 4 to get Papers regarding their Description
Enter your option: 2
Enter your query : TiMoTHY!?




The related Papers are:


paper_titles                                                                sonochemistry
authors                                             timothy james mason j phillip lorimer
publication_dates                                                               6/10/1905
descriptions                                                                          nan
Links                https://agris.fao.org/agris-search/search.do?recordID=US201300688309
Name: 0, dtype: object


paper_titles                                                                                                                                                                                                           


paper_titles                    honour professor timothy j mason
authors              muthupandian ashokkumar timothy james mason
publication_dates                                       7/9/1905
descriptions                                                 nan
Links                                                        NaN
Name: 246, dtype: object


paper_titles                                                                                                                                                                                                                                                                                                                                                                                                                                                  ultrasonically enabled low tempertaure electroless plating sustainable electronic manufacture
authors                                                                                                    